<a href="https://colab.research.google.com/github/AndresVillotaVillota/Competencia-Kaggle-sabrepro-2025-1/blob/main/03_modelo_con_preprocesado_y_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Selecciona el ZIP desde tu computador
uploaded = files.upload()

Saving udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip to udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip


In [2]:
import zipfile
import os

# Descomprimir el archivo ZIP
zip_path = "udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip"
extract_path = "datos_saberpro"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verifica que los archivos estén listos
print("Archivos extraídos:", os.listdir(extract_path))

Archivos extraídos: ['train.csv', 'submission_example.csv', 'test.csv']


In [3]:
import pandas as pd

# Leer archivos ya extraídos
train = pd.read_csv("datos_saberpro/train.csv")
test = pd.read_csv("datos_saberpro/test.csv")
sample_submission = pd.read_csv("datos_saberpro/submission_example.csv")

# Vista previa
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [5]:
# 02 - preprocesado.ipynb

# ✅ PREPROCESADO BÁSICO (Simple encoding y limpieza)
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Cargar datos
train = pd.read_csv("datos_saberpro/train.csv")
test = pd.read_csv("datos_saberpro/test.csv")
train["source"] = "train"
test["source"] = "test"
test["RENDIMIENTO_GLOBAL"] = np.nan

# Unificar
data = pd.concat([train, test], ignore_index=True)

# Imputar faltantes simples
for col in data.select_dtypes(include="number"):
    data[col] = data[col].fillna(data[col].median())

for col in data.select_dtypes(include="object"):
    data[col] = data[col].fillna("missing")

# Codificación label
for col in data.select_dtypes(include="object"):
    if col != "RENDIMIENTO_GLOBAL":
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])

# Separar
train_clean = data[data["source"] == "train"].drop(columns=["source"])
test_clean = data[data["source"] == "test"].drop(columns=["source", "RENDIMIENTO_GLOBAL"])

# Guardar datos preprocesados
train_clean.to_csv("datos_saberpro/train_clean.csv", index=False)
test_clean.to_csv("datos_saberpro/test_clean.csv", index=False)

In [ ]:
# 03 - modelo con preprocesado simple y SVM.ipynb

# ✅ SVM con preprocesado simple
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Cargar datos
train = pd.read_csv("datos_saberpro/train_clean.csv")
X = train.drop(columns=["RENDIMIENTO_GLOBAL", "ID"])
y = LabelEncoder().fit_transform(train["RENDIMIENTO_GLOBAL"])

# Train/test split local
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Entrenar modelo
clf = SVC(kernel="rbf", C=1.0, gamma="scale")
clf.fit(X_train, y_train)

# Validar
preds = clf.predict(X_val)
print("Accuracy:", accuracy_score(y_val, preds))